# OCR Read 'num2' Metadata

#### Updated: May 12, 2023

#  

Test post-processing a subdirectory, to first detect if the image is of 'num2' metadata type, and then to read and record that metadata. 

In [1]:
import pandas as pd
import numpy as np
import os
from random import randrange
import time
import keras_ocr

In [2]:
rootDir = 'L:/DATA/Alouette_I/BATCH_II_Run1/'
processedDir = rootDir + '04_processed/'
resultDir = rootDir + '05_result/'

In [3]:
pipeline = keras_ocr.pipeline.Pipeline()

Looking for C:\Users\rnaidoo\.keras-ocr\craft_mlt_25k.h5
Looking for C:\Users\rnaidoo\.keras-ocr\crnn_kurapan.h5


#  

#### Functions:

In [73]:
def read_num2_metadata(prediction_groups, subdir_path, batch_i, img_fns):
    
    df_read = pd.DataFrame()
    df_notread = pd.DataFrame()
    for i in range(0, len(prediction_groups)):
        df_ocr = pd.DataFrame()
        predicted_image = prediction_groups[i]
        if len(predicted_image) > 0:
            for text, box in predicted_image:
                row = pd.DataFrame({
                    'number': text,
                    'x': box[1][0],
                    'y': box[1][1]
                }, index=[0])
                df_ocr = pd.concat([df_ocr, row])
            df_ocr = df_ocr.sort_values('x').reset_index(drop=True)
        
            #String concatenate, fix string
            read_str = ''
            for j in range(0, len(df_ocr)):
                read_str_ = df_ocr['number'].iloc[j]
                read_str += read_str_
            read_str = read_str.replace('o', '0')

            #Test for num2
            if len(read_str) == 15:
                if read_str[0:2] == '10':
                    row2 = pd.DataFrame({
                        'station_number_OCR': read_str[2:4],
                        'year_OCR': read_str[4:6],
                        'day_of_year_OCR': read_str[6:9],
                        'hour_OCR': read_str[9:11],
                        'minute_OCR': read_str[11:13],
                        'second_OCR': read_str[13:15],
                        'filename': img_fns[batch_i + i].replace(subdir_path, '')
                    }, index=[i])
                    df_read = pd.concat([df_read, row2])
                else:
                    df_ocr['filename'] = img_fns[batch_i + i].replace(subdir_path, '')
                    df_notread = pd.concat([df_notread, df_ocr])
            else:
                df_ocr['filename'] = img_fns[batch_i + i].replace(subdir_path, '')
                df_notread = pd.concat([df_notread, df_ocr])
    
    return df_read, df_notread

# 

#### Process subdirectory:

In [76]:
#Draw random subdirectory
directory_list = os.listdir(processedDir)
directory = directory_list[randrange(len(directory_list))]
subdirectory_list = os.listdir(processedDir + directory + '/')
subdirectory = subdirectory_list[randrange(len(subdirectory_list))]
print(directory + '/' + subdirectory + '/')

R014207945/2266-13B/


In [77]:
#Manual subdirectory:
#directory = 'R014207817'
#subdirectory = '4163-03A'

#Manual subdirectory (num):
#directory = 'R014207941'
#subdirectory = '1765-5'

In [78]:
testDir = processedDir + directory + '/' + subdirectory + '/'

In [79]:
batch_size = 8 # > 9 seems to lead to a 'dead kernel'

In [80]:
img_fns = []
for file in os.listdir(testDir):
    img_fns.append(testDir + file)
len(img_fns)

354

In [81]:
n_batches = int(np.floor(len(img_fns)/batch_size))
n_batches

44

In [82]:
batch_remainder = len(img_fns)%batch_size
batch_remainder

2

In [83]:
start = time.time()

df_read = pd.DataFrame()
df_notread = pd.DataFrame()
for i in range(0, n_batches):
    print('Starting batch... ' + str(i))
    batch_i = i*batch_size
    batch_f = batch_i + batch_size
    prediction_groups = pipeline.recognize(img_fns[batch_i:batch_f])
    df_read_, df_notread_ = read_num2_metadata(prediction_groups=prediction_groups, subdir_path=testDir, batch_i=batch_i, 
                                               img_fns=img_fns)
    df_read = pd.concat([df_read, df_read_])
    df_notread = pd.concat([df_notread, df_notread_])
    
#Remainder
print('Finishing up...')
if batch_remainder > 0:
    batch_i = n_batches*batch_size
    batch_f = batch_i + batch_remainder
    prediction_groups = pipeline.recognize(img_fns[batch_i:batch_f])
    df_read_, df_notread_ = read_num2_metadata(prediction_groups=prediction_groups, subdir_path=testDir, batch_i=batch_i, 
                                              img_fns=img_fns)
    df_read = pd.concat([df_read, df_read_])
    df_notread = pd.concat([df_notread, df_notread_])
     
end = time.time()
t = end - start
print('Time to OCR read all images in subdirectory: ' + str(round(t/60, 1)) + ' min')

Starting batch... 0
1/1 [==============================] - 1s 568ms/step
Starting batch... 1
1/1 [==============================] - 1s 601ms/step
Starting batch... 2
1/1 [==============================] - 0s 469ms/step
Starting batch... 3
1/1 [==============================] - 1s 537ms/step
Starting batch... 4
1/1 [==============================] - 1s 763ms/step
Starting batch... 5
1/1 [==============================] - 1s 505ms/step
Starting batch... 6
1/1 [==============================] - 0s 368ms/step
Starting batch... 7
1/1 [==============================] - 1s 503ms/step
Starting batch... 8
1/1 [==============================] - 1s 646ms/step
Starting batch... 9
1/1 [==============================] - 1s 532ms/step
Starting batch... 10
1/1 [==============================] - 0s 355ms/step
Starting batch... 11
1/1 [==============================] - 0s 333ms/step
Starting batch... 12
1/1 [==============================] - 0s 280ms/step
Starting batch... 13
1/1 [======================

In [84]:
print(len(df_read))
df_read.head(30)

0


""


In [85]:
print(len(df_notread))
df_notread.head(30)

891


,number,x,y,filename
0,n,102.051270,341.413513,10.png
0,ns,141.009003,403.414429,100.png
1,la,1241.562500,359.625000,100.png
2,i,1340.031250,344.640625,100.png
0,ons,145.209641,400.052032,101.png
1,lwes,1359.606934,270.026672,101.png
2,ie,2053.859131,275.527405,101.png
3,,2069.812500,579.796875,101.png
4,paiin,2100.984375,440.562500,101.png
0,e,51.375000,254.734375,102.png


#  

#### Integrate OCR read metadata into existing results data for subdirectory:

In [104]:
df_result = pd.read_csv(resultDir + directory + '/' + 'result-' + directory + '_' + subdirectory + '.csv')
#Change 'Roll' to 'Directory':
df_result = df_result.rename(columns={
    'Roll': 'Directory'
})
print(len(df_result))
df_result.head()

354


,fmin,max_depth,day_1,day_2,day_3,hour_1,hour_2,minute_1,minute_2,second_1,...,limits,height,width,metadata_type,meta_height,meta_width,details,Directory,Subdirectory,filename
0,1.721774,1256.603774,1.0,8.0,1.0,8.0,10.0,2.0,3.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R014207941,1765-5,12.png
1,1.500000,1456.603774,1.0,8.0,1.0,8.0,10.0,8.0,1.0,17.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R014207941,1765-5,60.png
2,1.568548,1349.056604,4.0,1.0,4.0,9.0,1.0,2.0,2.0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R014207941,1765-5,10.png
3,1.500000,1349.056604,1.0,4.0,9.0,1.0,6.0,0.0,4.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R014207941,1765-5,100.png
4,1.536290,1220.754717,4.0,1.0,4.0,9.0,1.0,6.0,0.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R014207941,1765-5,101.png


In [105]:
df_result.columns

Index(['fmin', 'max_depth', 'day_1', 'day_2', 'day_3', 'hour_1', 'hour_2',
       'minute_1', 'minute_2', 'second_1', 'second_2', 'station_code',
       'station_number', 'processed_image_class', 'satellite_number', 'year',
       'station_number_1', 'station_number_2', 'func_name', 'limits', 'height',
       'width', 'metadata_type', 'meta_height', 'meta_width', 'details',
       'Directory', 'Subdirectory', 'filename'],
      dtype='object')

In [106]:
if len(df_result) > 0:
    if len(df_read) > 0:
        df_merge = df_result.merge(df_read, how='left', on='filename')
        for i in range(0, len(df_merge)):
            if df_merge['processed_image_class'].iloc[i] != 'loss':
                if df_merge['processed_image_class'].iloc[i] != 'outlier':
                    #if ~pd.isna(df_merge['day_of_year_OCR'].iloc[i]):
                    if int(df_merge['day_of_year_OCR'].iloc[i]) > 0:
                        df_merge['processed_image_class'].iloc[i] = 'num2'
        #df_merge.loc[~pd.isna(df_merge['day_of_year_OCR']), 'processed_image_class'] = 'num2'
        #df_merge.loc[df_merge['day_of_year_OCR'] > 0, 'processed_image_class'] = 'num2'
    else:
        df_merge = df_result
else:
    df_merge = df_result
    
print(len(df_merge))
df_merge.sample(10)

354


,fmin,max_depth,day_1,day_2,day_3,hour_1,hour_2,minute_1,minute_2,second_1,...,limits,height,width,metadata_type,meta_height,meta_width,details,Directory,Subdirectory,filename
42,1.556452,1349.056604,4.0,1.0,4.0,9.0,2.0,1.0,3.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R014207941,1765-5,140.png
156,1.556452,1349.056604,1.0,5.0,0.0,2.0,0.0,1.0,9.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R014207941,1765-5,251.png
291,1.754032,1315.094340,4.0,1.0,4.0,9.0,1.0,2.0,2.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R014207941,1765-5,6.png
327,1.512097,1315.094340,1.0,4.0,9.0,1.0,6.0,0.0,3.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R014207941,1765-5,96.png
123,1.741935,1394.339623,4.0,1.0,4.0,9.0,1.0,2.0,3.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R014207941,1765-5,22.png
311,1.564516,1256.603774,4.0,1.0,4.0,9.0,1.0,2.0,2.0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R014207941,1765-5,8.png
216,1.733871,1349.056604,1.0,5.0,1.0,1.0,3.0,5.0,0.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R014207941,1765-5,305.png
223,1.536290,1409.433962,4.0,1.0,5.0,1.0,1.0,3.0,5.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R014207941,1765-5,311.png
237,1.737903,1409.433962,4.0,1.0,5.0,1.0,1.0,5.0,3.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R014207941,1765-5,326.png
286,1.616935,1256.603774,4.0,1.0,4.0,9.0,1.0,4.0,1.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R014207941,1765-5,55.png


In [107]:
#df_merge[['day_of_year_OCR', 'processed_image_class']].sample(10)

Classify rows with OCR letters read as 'loss' and clear metadata:

In [108]:
OCR_cols = ['station_number_OCR', 'year_OCR', 'day_of_year_OCR', 'hour_OCR', 'minute_OCR', 'second_OCR']
md_cols = ['satellite_number', 'year', 'day_1', 'day_2', 'day_3', 'hour_1', 'hour_2', 'minute_1', 'minute_2', 'second_1', 
           'second_2', 'station_number_1', 'station_number_2']

In [109]:
if len(df_read) > 0:
    for col in OCR_cols:
        df_merge[col] = df_merge[col].astype('string')
        df_merge.loc[df_merge[col].str.contains("[a-zA-Z]"), 'processed_image_class'] = 'loss'
        df_merge.loc[df_merge[col].str.contains("[a-zA-Z]"), 'details'] = 'OCR read metadata contains letters'   

In [110]:
n_OCR_read = 0
for i in range(0, len(df_merge)):
    if df_merge['processed_image_class'].iloc[i] == 'loss':
        if df_merge['details'].iloc[i] == 'OCR read metadata contains letters':
            for col in md_cols:
                df_merge[col].iloc[i] = np.nan
    elif df_merge['processed_image_class'].iloc[i] == 'num2':
        for col in md_cols:
            df_merge[col].iloc[i] = np.nan
        n_OCR_read += 1

In [111]:
df_merge.loc[df_merge['processed_image_class'] == 'num']

,fmin,max_depth,day_1,day_2,day_3,hour_1,hour_2,minute_1,minute_2,second_1,...,limits,height,width,metadata_type,meta_height,meta_width,details,Directory,Subdirectory,filename
2,1.568548,1349.056604,4.0,1.0,4.0,9.0,1.0,2.0,2.0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R014207941,1765-5,10.png
3,1.500000,1349.056604,1.0,4.0,9.0,1.0,6.0,0.0,4.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R014207941,1765-5,100.png
4,1.536290,1220.754717,4.0,1.0,4.0,9.0,1.0,6.0,0.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R014207941,1765-5,101.png
5,1.500000,1071.698113,4.0,1.0,4.0,9.0,1.0,6.0,0.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R014207941,1765-5,102.png
6,1.520161,1315.094340,4.0,1.0,4.0,9.0,1.0,6.0,0.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R014207941,1765-5,103.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326,1.500000,749.056604,1.0,4.0,9.0,1.0,6.0,0.0,3.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R014207941,1765-5,95.png
327,1.512097,1315.094340,1.0,4.0,9.0,1.0,6.0,0.0,3.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R014207941,1765-5,96.png
328,1.508065,1256.603774,4.0,1.0,4.0,9.0,1.0,6.0,0.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R014207941,1765-5,97.png
329,1.536290,1220.754717,4.0,1.0,4.0,9.0,1.0,6.0,0.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R014207941,1765-5,98.png


If num2 metadata type is detected, classify images with all other metadata types as loss:

#### Assumption: film rolls are unlikely to have mixed metadata - assume that if num2 metadata type is detected, that there is no other images on the film roll with other types of metadata.

In [112]:
if len(df_read) > 0:
    for i in range(0, len(df_merge)):
        if df_merge['processed_image_class'].iloc[i] == 'num':
            df_merge['processed_image_class'].iloc[i] = 'loss'
            df_merge['details'].iloc[i] = 'metadata could not be read by OCR'
            for col in md_cols:
                df_merge[col].iloc[i] = np.nan
        if df_merge['processed_image_class'].iloc[i] == 'dot':
            df_merge['processed_image_class'].iloc[i] = 'loss'
            df_merge['details'].iloc[i] = 'metadata could not be read by OCR'
            for col in md_cols:
                df_merge[col].iloc[i] = np.nan

In [113]:
df_merge.loc[df_merge['processed_image_class'] == 'num']

,fmin,max_depth,day_1,day_2,day_3,hour_1,hour_2,minute_1,minute_2,second_1,...,limits,height,width,metadata_type,meta_height,meta_width,details,Directory,Subdirectory,filename
2,1.568548,1349.056604,4.0,1.0,4.0,9.0,1.0,2.0,2.0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R014207941,1765-5,10.png
3,1.500000,1349.056604,1.0,4.0,9.0,1.0,6.0,0.0,4.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R014207941,1765-5,100.png
4,1.536290,1220.754717,4.0,1.0,4.0,9.0,1.0,6.0,0.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R014207941,1765-5,101.png
5,1.500000,1071.698113,4.0,1.0,4.0,9.0,1.0,6.0,0.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R014207941,1765-5,102.png
6,1.520161,1315.094340,4.0,1.0,4.0,9.0,1.0,6.0,0.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R014207941,1765-5,103.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326,1.500000,749.056604,1.0,4.0,9.0,1.0,6.0,0.0,3.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R014207941,1765-5,95.png
327,1.512097,1315.094340,1.0,4.0,9.0,1.0,6.0,0.0,3.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R014207941,1765-5,96.png
328,1.508065,1256.603774,4.0,1.0,4.0,9.0,1.0,6.0,0.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R014207941,1765-5,97.png
329,1.536290,1220.754717,4.0,1.0,4.0,9.0,1.0,6.0,0.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R014207941,1765-5,98.png


In [114]:
if n_OCR_read > 0:
    print('Percent of OCR read images that read only numbers, for test subdirectory: ' + str(round((n_OCR_read/len(df_read))*100, 1)) + ' %')

#### If num2 metadata type is not detected:

In [115]:
if len(df_read) == 0:
    n_num = len(df_merge.loc[df_merge['processed_image_class'] == 'num'])
    n_dot = len(df_merge.loc[df_merge['processed_image_class'] == 'dot'])

In [116]:
n_num

329

In [117]:
n_dot

2

If num type metadata is the majority, classify dot type images as loss:

In [118]:
if n_num > n_dot:
    for i in range(0, len(df_merge)):
        if df_merge['processed_image_class'].iloc[i] == 'dot':
            df_merge['processed_image_class'].iloc[i] = 'loss'
            df_merge['details'].iloc[i] = 'metadata was interpreted to be dot type'
            for col in md_cols:
                df_merge[col].iloc[i] = np.nan

C:\Users\rnaidoo\AppData\Local\Temp\ipykernel_21704\4046991480.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merge['processed_image_class'].iloc[i] = 'loss'
C:\Users\rnaidoo\AppData\Local\Temp\ipykernel_21704\4046991480.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merge['details'].iloc[i] = 'metadata was interpreted to be dot type'
C:\Users\rnaidoo\AppData\Local\Temp\ipykernel_21704\4046991480.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retu

If dot type metadata is the majority, classify num type images as loss:

In [119]:
if n_dot > n_num:
    for i in range(0, len(df_merge)):
        if df_merge['processed_image_class'].iloc[i] == 'num':
            df_merge['processed_image_class'].iloc[i] = 'loss'
            df_merge['details'].iloc[i] = 'metadata was interpreted to be num type'
            for col in md_cols:
                df_merge[col].iloc[i] = np.nan   

#  

#### Save:

In [120]:
df_merge.to_csv(resultDir + directory + '/' + 'result_OCRpass-' + directory + '_' + subdirectory + '.csv', index=False)

#  

#### Development:

In [4]:
a = np.nan

In [5]:
b = 265

In [6]:
~np.isnan(a)

False

In [11]:
~pd.isna(a)

-2

In [15]:
pd.isna(a)

True

In [7]:
~np.isnan(b)

True

In [12]:
~pd.isna(b)

-1

In [16]:
pd.isna(b)

False

In [8]:
a + b

nan

In [17]:
if ~pd.isna(b):
    print('b is not NaN!')

b is not NaN!


In [18]:
if pd.isna(a):
    print('a is NaN!')

a is NaN!
